In [ ]:
import numpy as np 
import pandas as pd 
import glob
import os
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px

import plotly.graph_objects as go
from plotly.offline import iplot

In [ ]:
sub_746 = pd.read_csv('../input/scoredata/0.746.csv')
sub_674 = pd.read_csv('../input/tensorflow-score/submission.csv')
sub_667 = pd.read_csv('../input/scoredata/0.667.csv')

In [ ]:
sub_667

In [ ]:
hist_data = [sub_746.MGMT_value, sub_674.MGMT_value, sub_667.MGMT_value]
group_labels = ['sub_746', 'sub_674', 'sub_667']

fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False)

fig.update_layout(
    width = 700,
    height = 400
)
fig.show()

In [ ]:
# dict for the dataframes and their names
dfs = {"sub_746" : sub_746, "sub_674": sub_674, 'sub_667':sub_667}

# plot the data
fig = go.Figure()
for i in dfs:
    fig = fig.add_trace(go.Scatter(x = dfs[i]["BraTS21ID"],
                                   y = dfs[i]["MGMT_value"], 
                                   mode="markers",
                                   name = i))
    fig = fig.add_trace(
        go.Scatter(
            x=[0, 1010],
            y=[0.5, 0.5],
            mode="lines",
            line=go.scatter.Line(color="gray"),
            showlegend=False)
    )

fig.update_layout(
    width = 700,
    height = 500
)
fig.show()

In [ ]:
sub_746['MGMT_value'].values.mean()

In [ ]:
Finalsubmission = sub_746.copy()
Finalsubmission['MGMT_value'] =\
sub_746['MGMT_value'].values*0.40+\
sub_674['MGMT_value'].values*0.30+\
sub_667['MGMT_value'].values*0.30

In [ ]:
Finalsubmission['BraTS21ID'] = Finalsubmission['BraTS21ID'].apply(lambda x: str(x).zfill(5))
Fsubmission = Finalsubmission.set_index('BraTS21ID')
FsubmissionDict = Fsubmission['MGMT_value'].to_dict()

listOfStudyPaths = glob.glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/*')
listOfStudies = [eachPath.split('/')[-1] for eachPath in listOfStudyPaths]

predList = []
for eachStudy in listOfStudies:
    if eachStudy not in FsubmissionDict:
        predList.append('0.500')
    else:
        score = float(FsubmissionDict[eachStudy])
        predList.append(score)
        
submission = pd.DataFrame({'BraTS21ID':listOfStudies,'MGMT_value':predList})

In [ ]:
submission=submission.sort_values(by=['BraTS21ID'], ascending=True)
submission=submission.reset_index(drop=True)
submission.head(5)

In [ ]:
submission.to_csv('submission.csv', index=False)